In [8]:
from google.colab import drive
import pandas as pd
import os


# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Replace 'df.csv' with the actual path to your file in Google Drive if it's not in the root directory
file_path = '/content/drive/My Drive/df1.csv'

df1 = pd.read_csv(file_path, parse_dates=['time'])
df1 = df1.sort_values('time').reset_index(drop=True)


print(df1.head(3))
df1.dtypes

        time  GDP_4cum  city_mean_light  region_mean_light title website  \
0 2008-01-01   1634.65        12.553247           0.953015  (填补)    none   
1 2008-04-01   3346.63        13.280818           0.986813  (填补)    none   
2 2008-07-01   5351.74        13.191190           0.972665  (填补)    none   

   Year  Month      GDP    ln_gdp   ln_city  ln_region  city_prop  \
0  2008      1  1634.65  7.399184  2.529979  -0.048125     0.2381   
1  2008      4  1711.98  7.445406  2.586321  -0.013275     0.2519   
2  2008      7  2005.11  7.603454  2.579549  -0.027716     0.2502   

   region_prop   dln_gdp  dln_city  dln_region  d2ln_gdp  
0       0.2425       NaN       NaN         NaN       NaN  
1       0.2511  0.046222  0.056341    0.034850       NaN  
2       0.2475  0.158048 -0.006772   -0.014441  0.111826  


,0
time,datetime64[ns]
GDP_4cum,float64
city_mean_light,float64
region_mean_light,float64
title,object
website,object
Year,int64
Month,int64
GDP,float64
ln_gdp,float64


In [14]:
df1.shape

(69, 18)

In [16]:
df = df1[(df1['Year'] != 2012) & (df1['time'] != '2013-01-01')]
df.shape

(64, 18)

In [9]:
!pip install rasterio matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 97.1 MB/s eta 0:00:00


**Reasoning**:
Define and print the paths to the landsat5 and landsat7 folders in Google Drive.



In [17]:
# Define the path to the landsat5 folder in your Google Drive
landsat5_folder_path = '/content/drive/My Drive/landsat5'

# Define the path to the landsat7 folder in your Google Drive
landsat7_folder_path = '/content/drive/My Drive/landsat7'

# Print the paths to verify
print(f"Landsat 5 folder path: {landsat5_folder_path}")
print(f"Landsat 7 folder path: {landsat7_folder_path}")

Landsat 5 folder path: /content/drive/My Drive/landsat5
Landsat 7 folder path: /content/drive/My Drive/landsat7


**Reasoning**:
List files in both landsat folders, combine the lists, and sort them numerically based on the quarter number in the filename.



In [18]:

# List files in landsat5 folder
landsat5_files = [os.path.join(landsat5_folder_path, f) for f in os.listdir(landsat5_folder_path) if f.endswith('.tif')]

# List files in landsat7 folder
landsat7_files = [os.path.join(landsat7_folder_path, f) for f in os.listdir(landsat7_folder_path) if f.endswith('.tif')]

# Combine the lists
all_image_files = landsat5_files + landsat7_files

# Custom sorting key to extract the quarter number and sort numerically
def sort_key(file_path):
    filename = os.path.basename(file_path)
    quarter_str = filename.replace('Q', '').replace('.tif', '')
    try:
        return int(quarter_str)
    except ValueError:
        return float('inf') # Put files that don't match the pattern at the end

# Sort the combined list
all_image_files_sorted = sorted(all_image_files, key=sort_key)

# Display the first few and last few sorted file paths to verify
print("First 5 sorted image file paths:")
print(all_image_files_sorted[:5])
print("\nLast 5 sorted image file paths:")
print(all_image_files_sorted[-5:])

# Verify the total number of files matches the expected 60 quarters
print(f"\nTotal number of sorted image files: {len(all_image_files_sorted)}")

First 5 sorted image file paths:
['/content/drive/My Drive/landsat5/Q1.tif', '/content/drive/My Drive/landsat5/Q2.tif', '/content/drive/My Drive/landsat5/Q3.tif', '/content/drive/My Drive/landsat5/Q4.tif', '/content/drive/My Drive/landsat5/Q5.tif']

Last 5 sorted image file paths:
['/content/drive/My Drive/landsat7/Q60.tif', '/content/drive/My Drive/landsat5/p08-1.tif', '/content/drive/My Drive/landsat5/p08-3.tif', '/content/drive/My Drive/landsat5/p08-2.tif', '/content/drive/My Drive/landsat5/p08-4.tif']

Total number of sorted image files: 64


In [19]:
# Create a dictionary mapping time values to image paths
image_path_mapping = {
    '2008-01-01': '/content/drive/My Drive/landsat5/p08-1.tif',
    '2008-04-01': '/content/drive/My Drive/landsat5/p08-2.tif',
    '2008-07-01': '/content/drive/My Drive/landsat5/p08-3.tif',
    '2008-10-01': '/content/drive/My Drive/landsat5/p08-4.tif'
}

# Add the 'image_path' column to df, using the mapping.
# Use .loc to avoid SettingWithCopyWarning
df.loc[:, 'image_path'] = df['time'].astype(str).map(image_path_mapping)

# You can also fill any rows that didn't match the mapping with a default value like None or ''
# df.loc[:, 'image_path'] = df['image_path'].fillna('')

# Display the updated DataFrame to verify
print(df[['time', 'image_path']].head())

        time                                  image_path
0 2008-01-01  /content/drive/My Drive/landsat5/p08-1.tif
1 2008-04-01  /content/drive/My Drive/landsat5/p08-2.tif
2 2008-07-01  /content/drive/My Drive/landsat5/p08-3.tif
3 2008-10-01  /content/drive/My Drive/landsat5/p08-4.tif
4 2009-01-01                                         NaN


/tmp/ipython-input-19-2587083987.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'image_path'] = df['time'].astype(str).map(image_path_mapping)


In [21]:
# Update the dictionary with mappings for Q1 to Q12
# You can generate these mappings programmatically or list them explicitly
for i in range(1, 13):
    year = 2009 + (i - 1) // 4
    month = ((i - 1) % 4) * 3 + 1
    time_str = f'{year}-{month:02d}-01'
    image_path_mapping[time_str] = f'/content/drive/My Drive/landsat5/Q{i}.tif'

# Apply the updated mapping to the 'image_path' column,
# filling only the NaN values with the new mappings
df.loc[:, 'image_path'] = df['time'].astype(str).map(image_path_mapping).fillna(df['image_path'])

# Display the updated DataFrame to verify
print(df[['time', 'image_path']].head(17))

         time                                  image_path
0  2008-01-01  /content/drive/My Drive/landsat5/p08-1.tif
1  2008-04-01  /content/drive/My Drive/landsat5/p08-2.tif
2  2008-07-01  /content/drive/My Drive/landsat5/p08-3.tif
3  2008-10-01  /content/drive/My Drive/landsat5/p08-4.tif
4  2009-01-01     /content/drive/My Drive/landsat5/Q1.tif
5  2009-04-01     /content/drive/My Drive/landsat5/Q2.tif
6  2009-07-01     /content/drive/My Drive/landsat5/Q3.tif
7  2009-10-01     /content/drive/My Drive/landsat5/Q4.tif
8  2010-01-01     /content/drive/My Drive/landsat5/Q5.tif
9  2010-04-01     /content/drive/My Drive/landsat5/Q6.tif
10 2010-07-01     /content/drive/My Drive/landsat5/Q7.tif
11 2010-10-01     /content/drive/My Drive/landsat5/Q8.tif
12 2011-01-01     /content/drive/My Drive/landsat5/Q9.tif
13 2011-04-01    /content/drive/My Drive/landsat5/Q10.tif
14 2011-07-01    /content/drive/My Drive/landsat5/Q11.tif
15 2011-10-01    /content/drive/My Drive/landsat5/Q12.tif
21 2013-04-01 

In [23]:
# Continue updating the dictionary with mappings for Q13 to Q60
for i in range(13, 61):
    # Calculate the year and month for each Q number starting from 2013-04-01 (Q13)
    # Q13 is 2013-04-01
    # Q14 is 2013-07-01
    # Q15 is 2013-10-01
    # Q16 is 2014-01-01
    # and so on...

    # Base time for Q13
    base_year = 2013
    base_month = 4
    quarter_index_from_base = i - 13

    # Calculate the total number of months from the base time
    total_months_from_base = quarter_index_from_base * 3

    # Calculate the year and month
    year = base_year + (base_month - 1 + total_months_from_base) // 12
    month = ((base_month - 1 + total_months_from_base) % 12) + 1

    time_str = f'{year}-{month:02d}-01'
    image_path_mapping[time_str] = f'/content/drive/My Drive/landsat7/Q{i}.tif'

# Apply the updated mapping to the 'image_path' column,
# filling only the NaN values with the new mappings
df.loc[:, 'image_path'] = df['time'].astype(str).map(image_path_mapping).fillna(df['image_path'])

# Display the updated DataFrame to verify
print(df[['time', 'image_path']].tail())

         time                                image_path
64 2024-01-01  /content/drive/My Drive/landsat7/Q56.tif
65 2024-04-01  /content/drive/My Drive/landsat7/Q57.tif
66 2024-07-01  /content/drive/My Drive/landsat7/Q58.tif
67 2024-10-01  /content/drive/My Drive/landsat7/Q59.tif
68 2025-01-01  /content/drive/My Drive/landsat7/Q60.tif


In [26]:
# Select only the 'time' and 'image_path' columns
df_image = df[['time', 'image_path']]

# Define the path where you want to save the CSV file in your Google Drive
output_file_path = '/content/drive/My Drive/image.csv'

# Export the DataFrame to a CSV file
df_image.to_csv(output_file_path, index=False) # index=False prevents writing the DataFrame index as a column

print(f"DataFrame successfully exported to '{output_file_path}'")

DataFrame successfully exported to '/content/drive/My Drive/image.csv'
